In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
sns.boxplot(data["total_bedrooms"])

In [ ]:
data.describe()

In [ ]:
data["ocean_proximity"].value_counts()

In [ ]:
plt.figure(figsize = (12,8))
sns.heatmap(data.corr(), annot = True, cmap = 'YlGnBu')

In [ ]:
df = data.copy()
df.isnull().sum()

In [ ]:
df["total_bedrooms"].fillna(value = df["total_bedrooms"].median(), inplace = True)
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
dummies = pd.get_dummies(df["ocean_proximity"], drop_first = True)
dummies.head()

In [ ]:
df = pd.concat([df, dummies], axis = 1 )
df.head()

In [ ]:
df = df.drop(["ocean_proximity", "longitude", "latitude"], axis =1)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, train_size = 0.75, test_size = 0.25, random_state = 123)

In [ ]:
num_variables = ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population','households', 'median_income', 'median_house_value']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train[num_variables] = scaler.fit_transform(df_train[num_variables])
df_train.describe()

In [ ]:
y_train = df_train.pop('median_house_value')
X_train = df_train

In [ ]:
import statsmodels.api as sm

X_train_lm = sm.add_constant(X_train)

lr_1 = sm.OLS(y_train, X_train_lm).fit()
print(lr_1.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["Features"] = X_train.columns
vif["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X = X_train.drop('NEAR BAY', axis = 1)

X_train_lm = sm.add_constant(X)

lr_2 = sm.OLS(y_train, X_train_lm).fit()
print(lr_2.summary())

In [ ]:
vif = pd.DataFrame()
vif["Features"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X = X.drop('households', axis = 1)

X_train_lm = sm.add_constant(X)

lr_2 = sm.OLS(y_train, X_train_lm).fit()
print(lr_2.summary())

In [ ]:
vif = pd.DataFrame()
vif["Features"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X = X.drop('total_rooms', axis = 1)

X_train_lm = sm.add_constant(X)

lr_3 = sm.OLS(y_train, X_train_lm).fit()
print(lr_3.summary())

In [ ]:
vif = pd.DataFrame()
vif["Features"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X = X.drop('population', axis = 1)

X_train_lm = sm.add_constant(X)

lr_4 = sm.OLS(y_train, X_train_lm).fit()
print(lr_4.summary())

In [ ]:
vif = pd.DataFrame()
vif["Features"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
y_train_pred = lr_4.predict(X_train_lm)

fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
plt.title("Error Term", fontsize = 20)

In [ ]:
df_test[num_variables] = scaler.transform(df_test[num_variables])
df_test.describe()

In [ ]:
df_test = df_test.drop(["NEAR BAY", "households", "total_rooms", "population"], axis = 1)
df_test.describe()

In [ ]:
y_test = df_test.pop('median_house_value')
X_test = df_test

In [ ]:
X_test_lm = sm.add_constant(X_test)
y_test_pred = lr_4.predict(X_test_lm)

In [ ]:
plt.scatter(y_test, y_test_pred)
plt.title("Actual vs Predictions", fontsize = 20)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")